<a href="https://colab.research.google.com/github/anandkumar966396/GriD_Fashion/blob/master/evaluation_of_my_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
to

In [3]:
!pip install rouge_score # install rouge_score to avoid the error

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=b519d571f9c8fa8474bc0b2a3183c563836357b4225c11ed47a95e4e09961948
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import evaluate

# Load Pegasus for summarization
pegasus_tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")
pegasus_model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-xsum")
summarizer = pipeline("summarization", model=pegasus_model, tokenizer=pegasus_tokenizer)

# Load T5 paraphraser
paraphrase_tokenizer = AutoTokenizer.from_pretrained("ramsrigouthamg/t5_paraphraser")
paraphrase_model = AutoModelForSeq2SeqLM.from_pretrained("ramsrigouthamg/t5_paraphraser")

# Function: Summarize input text
def summarize_text(text):
    inputs = pegasus_tokenizer.encode(text, return_tensors="pt", truncation=True, max_length=1024)
    summary_ids = pegasus_model.generate(inputs, max_length=60, min_length=25, length_penalty=2.0, num_beams=5, early_stopping=True)
    return pegasus_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Function: Reword (paraphrase) summary
def paraphrase_text(text):
    input_text = f"paraphrase: {text} </s>"
    encoding = paraphrase_tokenizer.encode_plus(input_text, return_tensors="pt", max_length=256, truncation=True)
    outputs = paraphrase_model.generate(
        input_ids=encoding['input_ids'],
        attention_mask=encoding['attention_mask'],
        max_length=256,
        num_beams=5,
        num_return_sequences=1,
        temperature=1.3,
        top_k=50,
        top_p=0.95,
    )
    return paraphrase_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Load evaluation metric
rouge = evaluate.load("rouge")

# Example usage
if __name__ == "__main__":
    article = """
    The government of Telangana has introduced new agricultural reforms aimed at increasing crop yields and
    farmer income. These include subsidies for fertilizers and seeds, along with irrigation projects to improve
    water access in rural areas. The policies are expected to modernize farming and reduce economic distress
    among farmers.
    """

    print("🔍 Summarizing...")
    summary = summarize_text(article)
    print("📝 Summary:", summary)

    # Reference (expected) summary — you can also use human-generated summary here
    reference_summary = "Telangana's government introduces agricultural reforms including subsidies and irrigation projects to modernize farming and improve farmer income."

    print("\n📊 Evaluating with ROUGE...")
    result = rouge.compute(predictions=[summary], references=[reference_summary])
    print(result)


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


🔍 Summarizing...
📝 Summary: India's southern state of Telangana has introduced new agricultural reforms aimed at increasing crop yields and farmer income, as the BBC's Geeta Pandey reports.

📊 Evaluating with ROUGE...
{'rouge1': np.float64(0.3181818181818182), 'rouge2': np.float64(0.09523809523809526), 'rougeL': np.float64(0.27272727272727276), 'rougeLsum': np.float64(0.27272727272727276)}
